<a href="https://colab.research.google.com/github/hamagami/pycaret/blob/main/pycaret_anomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anomaly detection by PyCaret

PyCaretは AI開発において、データの前処理や可視化、モデル開発を数行のコードで実現可能なPythonのAutoMLライブラリです。異常検知についても代表的なアルゴリズムが用意されており、だれでもすぐにAIによる異常検知アプリケーションが開発できます。
**2025年9月時点では、GoogleColabで動作させる場合、ランタイムのバージョンをPython 3.1以前に戻す必要があります。**


knn, LOF, ABOD, HBOS, クラスタリング,部分空間法,PCA, MCD,Oneclass SVM, Isolation forest

pycaretをインストールします。
1分ほどかかります。他のモジュールとの依存関係について，いくつかエラーが表示されますが，今回の範囲では問題ありません，

In [ ]:
!pip install git+https://github.com/pycaret/pycaret.git@master --upgrade
%!pip install -q pycaret

実行に必要なモジュールをいくつか読み込みます。importで失敗する場合には、上記の通り、ランタイムのバージョンを最新から以前のもの（3.10より前）に設定してください。

In [ ]:
from pycaret.anomaly import *         #  PyCaretの異常検知関連のモジュールを読みこみます
from pycaret.datasets import get_data #  PyCaretが予め用意してくれいる色々なデータセットを取得するモジュールを読みこみます

import pandas as pd                   # pandasとは，表形式のデータを扱う便利なモジュールです　PyCaretの入出力はpandasのDataFrameと呼ぶ表形式のデータ型をつかいます
import matplotlib.pyplot as plt       # グラフを表示するモジュールを読みこみます

# pandasバージョン依存
pd.DataFrame.iteritems = pd.DataFrame.items

## 準備ができました。ここからPyCaretを実際に動かしていきます

### データの読み込み

まず，データを準備します。ここでは，PyCaretが予め演習用に用意してあるデータを使って基本的な動作をみています。次の行で， dataset という変数にデータが入ります。同時にdatasetに格納されたDataFrame形式の最初の数行が出力されます。

In [ ]:
dataset = get_data("anomaly")

上記のように，このデータは Col1, Col2, .... Col10という属性（カラム）からなる10次元のデータです。正常か異常かのラベルはありません，このデータの中でどのサンプルが外れているかを教師なしで学習します。上の行では最初の５行程度しか表示されていませんが，データ全体の行数，列数は datasetのshape属性で確認できます。shape属性をみると下記のように1000行10列の表であることがわかります。

In [ ]:
dataset.shape  # 1000行10列

本当はここで，各属性(カラム）の欠損を確認すべきところですが，今回は省略します。実際にはこのデータに欠損はありません。次に，各属性ごとのデータの分布をみておきます。分布を確認しなくてもこれ以降の処理は可能ですが，ある程度分布を押さえておくとトラブルシューティングの際に役立ちます。グラフを書くと，１～４程度のピークを持つ様々な分布を含んでいることがわかります。この程度のデータであっても，外れを直観的にみつけることは難しそうです。

In [ ]:
#以下の処理は理解できなくても問題ありません。各カラムごとに半透明のヒストグラムを重ねて書いています
for item, data in dataset.iteritems(): #１列ずつ取り出す
  plt.hist(data,bins=50,label=item,alpha=0.2)
  plt.legend()
  plt.show()

### セットアップ

PyCaretでは，面倒なデータの前処理を自動化するしくみがあります。これをsetupといいます。setupには，前処理のための様々なオプションがあります。ここでは，元のデータが0～1に規格化されているため，改めて標準化（各属性の分布を平均0,分散1にしまｓ）を行わないようにしています。予めデータの分布を確認して，必要に応じて標準化，正規化をするようにしてください。またデータ全体に占める異常の割合を3%と設定しています。このパラメータはあとでまた再検討します。setupを行うと，データに関する情報・操作内容の一覧が出力されます。

In [ ]:
anoexe = setup(dataset,normalize=False,outliers_threshold=0.05) #データの範囲はほぼ0～1に収まっているので特段前処理を行わない設定にしている

準備が整ったので，異常検知をためしてみます。
どのような異常検知アルゴリズムが使えるかは，models()で確認できます。

In [ ]:
models()

**以下ではこれらのアルゴリズムを実行します。**

knn, LOF, ABOD, HBOS, クラスタリング,部分空間法,PCA, MCD,Oneclass SVM, Isolation forest



---



---

## **knn**


knnの学習モデルをつくります

Create a learning model for knn.

In [ ]:
knn_model = create_model('knn')

knnの学習を行います


learning knn model

In [ ]:
knn_results = assign_model(knn_model)

得られた結果を簡易的にグラフにします。横軸にデータ番号(0～999) 縦軸が異常度です。自動で設定された閾値に基づいて異常判定を行った結果を赤色の帯で表しています。

The obtained results are simply graphed. The horizontal axis is the data number (0 to 999) and the vertical axis is the abnormality level. The results of the abnormality judgment based on the automatically set threshold values are shown in light blue bands.

In [ ]:
plt.plot(knn_results["Anomaly_Score"],label="score")
plt.plot(knn_results["Anomaly"],alpha=0.5,label="results",color="red")
plt.title("knn")
plt.xlabel("data no")
plt.ylabel("anomaly score")
plt.legend()
plt.show()

異常度の分布を確認しおきます。水色が正常判定されたAnomaly_Score，赤が異常判定されたAnomaly_Scoreです。setupで全体の集合の5%が外れているという仮定で閾値がされているため，上位50までの外れが外れ値にしています。
このヒストグラムをみて，分布に基づく閾値（ネイマンピアソン決定則）を適用してもいいです。

The distribution of abnormality is shown below. Light blue is the normalized Anomaly_Score and orange is the abnormalized Anomaly_Score.
You can look at this histogram and apply a threshold based on the distribution (Neyman-Pearson decision rule).

In [ ]:
plt.hist(knn_results[knn_results["Anomaly"]==0]["Anomaly_Score"],bins=50,alpha=0.5,label="normal")
plt.hist(knn_results[knn_results["Anomaly"]==1]["Anomaly_Score"],bins=50,alpha=0.5,color="red",label="anomaly")
plt.title("knn outlier histgram")
plt.xlabel("anomaly score")
plt.ylabel("freq")
plt.legend()
plt.show()



---
## **局所外れ因子　Local Outlier Factor: LOF**



knnと同様にモデルの生成，学習を行います

In [ ]:
lof_model = create_model('lof')
lof_results = assign_model(lof_model)

異常度と判定結果を描画します。
異常度の値の範囲がknnとは異なりますが，ほぼ同じ個所で異常が検出できていることがわかります

In [ ]:
plt.plot(lof_results["Anomaly_Score"],label="score")
plt.plot(lof_results["Anomaly"],alpha=0.5,label="results",color="red")
plt.title("lof")
plt.xlabel("data no")
plt.ylabel("anomaly score")
plt.legend()
plt.show()

異常度のヒストグラムをとると，knnに比べて正常データの凝集がより強くなっており，異常検知が容易になっていることがわかります。こちらもこの分布をもとに統計的な意味をもった閾値を再設定をすると，より厳密な異常検知となります。

In [ ]:
plt.hist(lof_results[lof_results["Anomaly"]==0]["Anomaly_Score"],bins=50,alpha=0.5,label="normal")
plt.hist(lof_results[lof_results["Anomaly"]==1]["Anomaly_Score"],bins=50,alpha=0.5,color="red",label="anomaly")
plt.title("lof outlier histgram")
plt.xlabel("anomaly score")
plt.ylabel("freq")
plt.legend()
plt.show()

## **ABOD**

他のアルゴリズムについても同様にみていきます

In [ ]:
abod_model = create_model('abod')
abod_results = assign_model(abod_model)

In [ ]:
plt.plot(abod_results["Anomaly_Score"]/50,label="score")# 異常度のレンジが大きいため 1/50にして表示
plt.plot(abod_results["Anomaly"],alpha=0.5,label="results",color="red")
plt.title("ABOD")
plt.xlabel("data no")
plt.ylabel("anomaly score(×50)")
plt.legend()
plt.show()

異常度の分布でみると，異常データは0付近の大変狭い領域内に凝集しているが，すぐそばに正常データも多くあり，正常と異常の境界が重なっていることがわかる。正常と異常の分離はできているが，このデータは10次元のため高次元に強いABODのメリットがあまり生かされていないと考えられる

In [ ]:
plt.hist(abod_results[abod_results["Anomaly"]==0]["Anomaly_Score"],bins=50,alpha=0.5,label="normal")
plt.hist(abod_results[abod_results["Anomaly"]==1]["Anomaly_Score"],bins=1,alpha=0.5,color="red",label="anomaly")
plt.title("ABOD outlier histgram")
plt.xlabel("anomaly score")
plt.ylabel("freq")
plt.legend()
plt.show()



---

# **HBOS**

In [ ]:
hbos_model = create_model('histogram')
hbos_results = assign_model(hbos_model)

In [ ]:
plt.plot(hbos_results["Anomaly_Score"]/5,label="score") #レンジを合わせるため1/5している
plt.plot(hbos_results["Anomaly"],alpha=0.5,label="results",color="red")
plt.title("HBOS")
plt.xlabel("data no")
plt.ylabel("anomaly score(×5)")
plt.legend()
plt.show()

In [ ]:
plt.hist(hbos_results[hbos_results["Anomaly"]==0]["Anomaly_Score"],bins=50,alpha=0.5,label="normal")
plt.hist(hbos_results[hbos_results["Anomaly"]==1]["Anomaly_Score"],bins=50,alpha=0.5,color="red",label="anomaly")
plt.title("HBOS outlier histgram")
plt.xlabel("anomaly score")
plt.ylabel("freq")
plt.legend()
plt.show()



---

# **クラスタリング**

In [ ]:
clust_model = create_model('cluster')
clust_results = assign_model(clust_model)

In [ ]:
plt.plot(clust_results["Anomaly_Score"],label="score")
plt.plot(clust_results["Anomaly"],alpha=0.5,label="results",color="red")
plt.title("clustering")
plt.xlabel("data no")
plt.ylabel("anomaly score(×5)")
plt.legend()
plt.show()

In [ ]:
plt.hist(clust_results[clust_results["Anomaly"]==0]["Anomaly_Score"],bins=50,alpha=0.5,label="normal")
plt.hist(clust_results[clust_results["Anomaly"]==1]["Anomaly_Score"],bins=50,alpha=0.5,color="red",label="anomaly")
plt.title("Clustering")
plt.xlabel("anomaly score")
plt.ylabel("freq")
plt.legend()
plt.show()



---
# **部分空間法**


In [ ]:
sod_model = create_model('sod')
sod_results = assign_model(sod_model)

In [ ]:
plt.plot(sod_results["Anomaly_Score"],label="score")
plt.plot(sod_results["Anomaly"],alpha=0.5,label="results",color="red")
plt.title("sod: Subspace Outlier Detection")
plt.xlabel("data no")
plt.ylabel("anomaly score")
plt.legend()
plt.show()

In [ ]:
plt.hist(sod_results[sod_results["Anomaly"]==0]["Anomaly_Score"],bins=50,alpha=0.5,label="normal")
plt.hist(sod_results[sod_results["Anomaly"]==1]["Anomaly_Score"],bins=50,alpha=0.5,color="red",label="anomaly")
plt.title("sod: Subspace Outlier Detection")
plt.xlabel("anomaly score")
plt.ylabel("freq")
plt.legend()
plt.show()



---
# **PCA, MCD**


In [ ]:
pca_model = create_model('pca')
pca_results = assign_model(pca_model)

PCAは正常・異常関係なく直交空間をつくっているため，異常値の影響をうけてしまいます

In [ ]:
plt.plot(pca_results["Anomaly_Score"]/800,label="score")
plt.plot(pca_results["Anomaly"],alpha=0.5,label="results",color="red")
plt.title("PCA")
plt.xlabel("data no")
plt.ylabel("anomaly score (×800)")
plt.legend()
plt.show()

In [ ]:
plt.hist(pca_results[pca_results["Anomaly"]==0]["Anomaly_Score"],bins=50,alpha=0.5,label="normal")
plt.hist(pca_results[pca_results["Anomaly"]==1]["Anomaly_Score"],bins=50,alpha=0.5,color="red",label="anomaly")
plt.title("PCA")
plt.xlabel("anomaly score")
plt.ylabel("freq")
plt.legend()
plt.show()

In [ ]:
mcd_model = create_model('mcd')
mcd_results = assign_model(mcd_model)


In [ ]:
plt.plot(mcd_results["Anomaly_Score"]/100,label="score")
plt.plot(mcd_results["Anomaly"],alpha=0.5,label="results",color="red")
plt.title("MCD:Minimum Covariance Determinant")
plt.xlabel("data no")
plt.ylabel("anomaly score (×100)")
plt.legend()
plt.show()

In [ ]:
plt.hist(mcd_results[mcd_results["Anomaly"]==0]["Anomaly_Score"],bins=50,alpha=0.5,label="normal")
plt.hist(mcd_results[mcd_results["Anomaly"]==1]["Anomaly_Score"],bins=50,alpha=0.5,color="red",label="anomaly")
plt.title("MCD:Minimum Covariance Determinant")
plt.xlabel("anomaly score")
plt.ylabel("freq")
plt.legend()
plt.show()



---
# **one class SVM**


In [ ]:
svm_model = create_model('svm', kernel="rbf", fraction=0.01)
svm_results = assign_model(svm_model)


In [ ]:
plt.plot(svm_results["Anomaly_Score"]/30,label="score")
plt.plot(svm_results["Anomaly"],alpha=0.5,label="results",color="red")
plt.title("one class SVM")
plt.xlabel("data no")
plt.ylabel("anomaly score (x30)")
plt.legend()
plt.show()



---

# **isolation forest**

In [ ]:
if_model = create_model('iforest', fraction=0.01)
if_results = assign_model(if_model)


In [ ]:
plt.plot(if_results["Anomaly_Score"]*10,label="score")
plt.plot(if_results["Anomaly"],alpha=0.5,label="results",color="red")
plt.title("one class SVM")
plt.xlabel("data no")
plt.ylabel("anomaly score (/10)")
plt.legend()
plt.show()